In [14]:
import pandas as pd
import numpy as np

In [55]:
df = pd.read_csv("data/stream_e.csv", sep = "ef5r64i", engine = "python")
print(df.columns, df.shape)


Index(['Texto', 'IdUsuario', 'FechaCreacion', 'IdTweet', 'NombreUsuario',
       'FechaCreacionTweet', 'Hashtags', 'Url'],
      dtype='object') (1268, 8)


In [5]:
#Creating the dict with unique users and a list for the tweets they have written
corpus = {}
for i in range(df.shape[0]):
    if df.iloc[i]["NombreUsuario"] in corpus:
        corpus[df.iloc[i]["NombreUsuario"]].append(df.iloc[i]["Texto"])
    else:
        corpus[df.iloc[i]["NombreUsuario"]] = [df.iloc[i]["Texto"]]
type(corpus), len(corpus)

(dict, 1189)

In [70]:
documents = list(df.Texto)

In [73]:
#LSA

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
  
# raw documents to tf-idf matrix: 
vectorizer = TfidfVectorizer(stop_words='english', 
                             use_idf=True, 
                             smooth_idf=True)
# SVD to reduce dimensionality: 
svd_model = TruncatedSVD(n_components=100,         # num dimensions
                         algorithm='randomized',
                         n_iter=10)
# pipeline of tf-idf + SVD, fit to and applied to documents:
svd_transformer = Pipeline([('tfidf', vectorizer), 
                            ('svd', svd_model)])
svd_matrix = svd_transformer.fit_transform(documents)

# svd_matrix can later be used to compare documents, compare words, or compare queries with documents

In [98]:
#LDA

import gensim

from gensim.corpora.Dictionary import load_from_text, doc2bow
from gensim.corpora import MmCorpus
from gensim.models.ldamodel import LdaModel
document = "This is some document..."
# load id->word mapping (the dictionary)
id2word = load_from_text('wiki_en_wordids.txt')
# load corpus iterator
mm = MmCorpus('wiki_en_tfidf.mm')
# extract 100 LDA topics, updating once every 10,000
lda = LdaModel(corpus=mm, id2word=id2word, num_topics=100, update_every=1, chunksize=10000, passes=1)
# use LDA model: transform new doc to bag-of-words, then apply lda
doc_bow = doc2bow(document.split())
doc_lda = lda[doc_bow]
# doc_lda is vector of length num_topics representing weighted presence of each topic in the doc

ModuleNotFoundError: No module named 'gensim.corpora.Dictionary'

---------

In [51]:
import pyodbc 
import configLocal

server = configLocal.server
database = configLocal.database
username = configLocal.username
password = configLocal.password
cnxn = configLocal.cnxn
cursor = cnxn.cursor()

In [175]:
data = pd.read_sql_query("select msg_id, msg_users_id, msg_timestamp, msg_text from dbo.msg", cnxn)

In [181]:
data.head()

,msg_id,msg_users_id,msg_timestamp,msg_text
0,1123283650638942211,428308122,1556646666570,@aritzeta @bibiloni @JosepMClD @Ratafia_Party ...
1,1123283654740983808,1669818008,1556646667548,Que fuerte!!! Y que lastima
2,1123283664123703296,2295120738,1556646669785,"@Helga_tito Bro no me quejo, si me parece de p..."
3,1123283666682220544,3327214991,1556646670395,@iescolar Evidente!!!
4,1123283688136032256,141861159,1556646675510,Karen me manda msj todos los días ??


---------

In [203]:
corpus = []

for i in range(len(data)):
    corpus.append(data.iloc[i]["msg_text"])

In [183]:
import gensim
import logging
import tempfile
import os

TEMP_FOLDER = tempfile.gettempdir()
print('Folder "{}" will be used to save temporary dictionary and corpus.'.format(TEMP_FOLDER))

from gensim import corpora
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Folder "C:\Users\alonsoga\AppData\Local\Temp" will be used to save temporary dictionary and corpus.


In [204]:
from nltk.corpus import stopwords
from string import punctuation

# remove common words and tokenize
list1 = ['RT','rt']
stoplist = stopwords.words('spanish') + list(punctuation) + list1

texts = [[word for word in str(document).lower().split() if word not in stoplist] for document in corpus]

In [185]:
dictionary = corpora.Dictionary(texts)
dictionary.save(os.path.join(TEMP_FOLDER, 'elon.dict'))  # store the dictionary, for future reference
print(dictionary)

Dictionary(55378 unique tokens: ['@aritzeta', '@bibiloni', '@jbadia16', '@josepmcld', '@ratafia_party']...)


In [186]:
print(dictionary.token2id)

{'@aritzeta': 0, '@bibiloni': 1, '@jbadia16': 2, '@josepmcld': 3, '@ratafia_party': 4, 'diria': 5, 'https://t.co/25w8rz6vlq': 6, 'https://t.co/uhxfhidu0n': 7, 'jo': 8, 'però': 9, 'reflexiu': 10, 'sobrer:': 11, 'també.': 12, 'és': 13, 'fuerte!!!': 14, 'lastima': 15, '@helga_tito': 16, 'bro': 17, 'casters': 18, 'comentaba.': 19, 'lvp.': 20, 'madre': 21, 'mejores': 22, 'parece': 23, 'puta': 24, 'quejo,': 25, 'si': 26, 'solo': 27, '@iescolar': 28, 'evidente!!!': 29, '??': 30, 'días': 31, 'karen': 32, 'manda': 33, 'msj': 34, '40%': 35, '??los': 36, '@autonomosata': 37, '@lorenzoamor_ata': 38, 'alertan': 39, 'autónomos': 40, 'año': 41, 'creación': 42, 'desplomará': 43, 'empleo': 44, 'https://t.co/gixdu7usml': 45, 'https://t.co/mzzdyw28ap': 46, '@supersonia99': 47, 'jajaja': 48, 'madurao': 49, ':(': 50, 'quiero': 51, '@gobysimon': 52, 'ahaha': 53, 'currently': 54, 'doing': 55, 'get': 56, 'i’m': 57, 'perfectly': 58, 'same.': 59, 'the': 60, 'you': 61, '@conchetta9': 62, '@educitof': 63, 'engañó

In [187]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize(os.path.join(TEMP_FOLDER, 'elon.mm'), corpus)  # store to disk, for later use

In [188]:
from gensim import corpora, models, similarities


In [189]:
tfidf = models.TfidfModel(corpus) # step 1 -- initialize a model


In [190]:
corpus_tfidf = tfidf[corpus]  # step 2 -- use the model to transform vectors


LDA

In [191]:
total_topics = 10


In [192]:
lda = models.LdaModel(corpus, id2word=dictionary, num_topics=total_topics)
corpus_lda = lda[corpus_tfidf] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

In [193]:
#Show first n important word in the topics:
lda.show_topics(total_topics,10)

[(0,
  '0.009*"quiero" + 0.007*"si" + 0.006*"aún" + 0.006*"xd" + 0.006*"día" + 0.005*"trabajar" + 0.005*"juego" + 0.005*"clase" + 0.005*"ver" + 0.004*"hora"'),
 (1,
  '0.018*"????" + 0.013*"??????" + 0.010*"gracias" + 0.009*"????????????" + 0.009*"barcelona," + 0.007*"spain" + 0.007*"????????" + 0.006*"feliz" + 0.006*"día" + 0.005*"??????????"'),
 (2,
  '0.007*"va" + 0.006*"ayer" + 0.006*"vamos" + 0.005*"bien." + 0.005*"saber" + 0.005*"gol" + 0.005*"si" + 0.005*"vida" + 0.005*"gafas" + 0.004*"manera"'),
 (3,
  '0.013*"q" + 0.009*"…" + 0.007*"nivel" + 0.007*"barcelona" + 0.006*"madre" + 0.006*"ahora" + 0.005*"vergüenza" + 0.005*"mejor" + 0.005*"puta" + 0.005*"#dgt"'),
 (4,
  '0.011*"@krls" + 0.008*"foto" + 0.007*"@boye_g" + 0.007*"acaba" + 0.007*"nunca" + 0.006*"muchas" + 0.006*"!!!" + 0.005*"pude" + 0.005*"publicar" + 0.005*"debería"'),
 (5,
  '0.010*"sistema" + 0.007*"ve" + 0.006*"4" + 0.006*"dia" + 0.005*"via" + 0.004*"besos" + 0.004*"tras" + 0.004*"experiencia" + 0.004*"fin" + 0.004

In [166]:
from collections import OrderedDict

data_lda = {i: OrderedDict(lda.show_topic(i,25)) for i in range(total_topics)}
#data_lda

In [167]:
import pandas as pd

df_lda = pd.DataFrame(data_lda)
print(df_lda.shape)
df_lda = df_lda.fillna(0).T
print(df_lda.shape)

(25, 1)
(1, 25)


In [168]:
df_lda

,#camilitarenuncie,(a.d.l.e),@zuluagacamila,bien,estã¡,estã¡n,gente,gobierno,gracias,haber,...,nule,paã­s,presidente,puede,q,quã©,s,si,vida,â
0,0.001162,0.001517,0.001323,0.001291,0.001291,0.00113,0.001291,0.001614,0.001098,0.001097,...,0.001516,0.001162,0.001776,0.001129,0.002937,0.001388,0.001807,0.003872,0.001388,0.002001


In [171]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
panel = pyLDAvis.gensim.prepare(lda, corpus_lda, dictionary, mds='tsne')
panel

C:\ProgramData\Anaconda3\lib\site-packages\past\types\oldstr.py:33: DeprecationWarning: invalid escape sequence \d
  """
C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


ValueError: Found array with 1 sample(s) (shape=(1, 1)) while a minimum of 2 is required.

---------

extract entities

In [195]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

In [196]:
ex = 'European authorities fined Google a record $5.1 billion on Wednesday for abusing its power in the mobile phone market and ordered the company to alter its practices'

In [220]:
def preprocess(sent):
    sent = nltk.word_tokenize(sent, language="spanish")
    sent = nltk.pos_tag(sent, lang="spanish")
    return sent

In [221]:
sent = preprocess(ex)
sent

NotImplementedError: Currently, NLTK pos_tag only supports English and Russian (i.e. lang='eng' or lang='rus')

In [ ]:
ex = str([word for word in corpus])

In [ ]:
preprocess(corpus[0])

https://stackoverflow.com/questions/14732465/nltk-tagging-spanish-words-using-a-corpus  
    https://www.quora.com/Which-languages-are-available-in-nltk-part-of-speech-tagger